<a href="https://colab.research.google.com/github/nmh-dee/Safety-Clothing-Detection-Yolov3tiny-RaspberryPi/blob/main/Train_Yolov3tiny.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Step 1. Mount drive 
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
%cd /content/gdrive/My\ Drive/darknet
!unrar e data.rar

In [ ]:
#Step 2. Tai ma nguon YOLO ve drive
#Step 3: Upload file Data, yolov3-tiny.cfg,Makefile, len Google Drive

# Step 4. Tạo file yolo.names
# Từ dòng 2 là >> 

%cd /content/gdrive/My\ Drive/darknet
!echo "wearning a hard hat" > yolo.names
!echo "not wearing a hard hat" >> yolo.names
!echo "wearing a safety vest" >> yolo.names
!echo "not wearing a safety vest" >> yolo.names
!echo "wearing a mask" >> yolo.names
!echo "not wearing a mask" >> yolo.names


/content/gdrive/MyDrive/darknet


In [ ]:
# Step 5. Tạo file train.txt và val.txt

%cd /content/gdrive/My\ Drive/darknet
 
import glob2
import math  
import os
import numpy as np
 
files = []
for ext in ["*.png", "*.jpeg", "*.jpg"]:
  image_files = glob2.glob(os.path.join("data", ext))
  files += image_files
 
nb_val = math.floor(len(files)*0.1)
rand_idx1 = np.random.randint(0, 2000, nb_val)
rand_idx2 = np.random.randint(2100,4000, nb_val)

 
# Tạo file train.txt
with open("train.txt", "w") as f:
  for idx in np.arange(len(files)):
    if (idx not in rand_idx1) and (idx not in rand_idx2) and (os.path.exists(files[idx][:-3] + "txt")):
      f.write(files[idx]+'\n')
 
# Tạo file vali.txt
with open("val.txt", "w") as f:
  for idx in np.arange(len(files)):
    if (idx in rand_idx1) and (os.path.exists(files[idx][:-3] + "txt")):
      f.write(files[idx]+'\n')
# Tạo file vali.txt
with open("test.txt", "w") as f:
  for idx in np.arange(len(files)):
    if (idx in rand_idx2) and (os.path.exists(files[idx][:-3] + "txt")):
      f.write(files[idx]+'\n')

/content/gdrive/My Drive/darknet


In [ ]:
# Step 6. Tạo file yolo.data

%cd /content/gdrive/My\ Drive/darknet/darknet
!mkdir backup
!echo classes=6 > yolo.data    # Sửa 1 thành số class
!echo train=train.txt >> yolo.data
!echo valid=val.txt >> yolo.data
!echo names=yolo.names >> yolo.data
!echo backup=backup >> yolo.data

/content/gdrive/My Drive/darknet/darknet
mkdir: cannot create directory ‘backup’: File exists


In [ ]:
#take a look at the kind of GPU we have
!nvidia-smi

Sat Dec 19 03:44:35 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8    11W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Change the number depending on what GPU is listed above, under NVIDIA-SMI > Name.
# Tesla K80: 30
# Tesla P100: 60
# Tesla T4: 75


In [ ]:
# Step 7. Make darknet
%cd /content/gdrive/My\ Drive/darknet/darknet
%env compute_capability=75
!sed -i 's/OPENCV=0/OPENCV=1/g' Makefile
!sed -i 's/GPU=0/GPU=1/g' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/g' Makefile
!sed -i "s/ARCH= -gencode arch=compute_75,code=sm_75/ARCH= -gencode arch=compute_${compute_capability},code=sm_${compute_capability}/g" Makefile
!make

In [ ]:
# Step 9. Train
%cd /content/gdrive/MyDrive/darknet/darknet/
#!./darknet detector train yolo.data cfg/yolov4-tiny.cfg yolov4-tiny.conv.29 -dont_show -map
!./darknet detector train yolo.data cfg/yolov3-tiny.cfg yolov3-tiny.weights -dont_show -map -clear 


In [ ]:
%cd /content/gdrive/MyDrive/darknet/darknet

!./darknet detector map yolo.data cfg/yolov3-tiny.cfg ./backup/yolov3-tiny_best.weights

/content/gdrive/MyDrive/darknet/darknet
 CUDA-version: 10010 (10010), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
 0 : compute_capability = 750, cudnn_half = 0, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 1, batch = 8, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 conv     16       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  16 0.150 BF
   1 max                2x 2/ 2    416 x 416 x  16 ->  208 x 208 x  16 0.003 BF
   2 conv     32       3 x 3/ 1    208 x 208 x  16 ->  208 x 208 x  32 0.399 BF
   3 max                2x 2/ 2    208 x 208 x  32 ->  104 x 104 x  32 0.001 BF
   4 conv     64       3 x 3/ 1    104 x 104 x  32 ->  104 x 104 x  64 0.399 BF
   5 max                2x 2/ 2    104 x 104 x  64 ->   52 x  52 x  64 0.001 BF
   6 conv    128       3 x 3/ 1     52 x  52 x  64 ->   52 x  52 x 128 0.399 BF
   7 max                2x 2/ 2     52 x  52 x 128 ->   26 x  26 x 128 0.000 BF
   8 conv    256       3 x 3

In [ ]:
%cd /content/gdrive/MyDrive/darknet/darknet

!./darknet detector map yolo.data cfg/yolov4-tiny.cfg ./backup/yolov4-tiny_best.weights

/content/gdrive/MyDrive/darknet/darknet
 CUDA-version: 10010 (10010), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
 0 : compute_capability = 750, cudnn_half = 0, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 1, batch = 8, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 2    416 x 416 x   3 ->  208 x 208 x  32 0.075 BF
   1 conv     64       3 x 3/ 2    208 x 208 x  32 ->  104 x 104 x  64 0.399 BF
   2 conv     64       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x  64 0.797 BF
   3 route  2 		                       1/2 ->  104 x 104 x  32 
   4 conv     32       3 x 3/ 1    104 x 104 x  32 ->  104 x 104 x  32 0.199 BF
   5 conv     32       3 x 3/ 1    104 x 104 x  32 ->  104 x 104 x  32 0.199 BF
   6 route  5 4 	                           ->  104 x 104 x  64 
   7 conv     64       1 x 1/ 1    104 x 104 x  64 ->  104 x 104 x  64 0.089 BF
   8 route  2 7 	                           ->  104 x 104 x